# AMP-SCZ MRIQC classification

In [ ]:
import bids
import pandas as pd
import numpy as np
from matplotlib import pyplot

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
T1w_train = pd.read_csv('T1w_train.csv', index_col=0)
T2w_train = pd.read_csv('T2w_train.csv', index_col=0)
BOLD_train = pd.read_csv('BOLD_train.csv', index_col=0)
T1w_test = pd.read_csv('T1w_test.csv', index_col=0)
T2w_test = pd.read_csv('T2w_test.csv', index_col=0)
BOLD_test = pd.read_csv('BOLD_test.csv', index_col=0)

In [ ]:
BOLD_train['pass'].value_counts()

In [ ]:
print(T1w_train.shape)

In [ ]:
T1w_train

In [ ]:
T1w_train.corrwith(T1w_train.rating).dropna().sort_values()

In [ ]:
T1w_train.corrwith(T1w_train["pass"]).dropna().sort_values()

In [ ]:
def find_optimal_regressor(dataframe, dependent_variable_name, also_drop=()):
    # Separate independent and dependent variables
    X = dataframe.drop([dependent_variable_name] + list(also_drop), axis=1)
    y = dataframe[dependent_variable_name]
    
    # Define pipeline with standardization and regression model
    pipe = Pipeline([
        ('scale', StandardScaler()),
        ('model', LinearRegression())
    ])
    
    # Define hyperparameters for grid search
    linear_param_grid = {
        'model': [LinearRegression()],
    }
    
    forest_param_grid = {
        'model': [RandomForestRegressor()],
        'model__n_estimators': [10, 50, 100],
        'model__max_depth': [None, 5, 10],
    }
    
    # Define nested cross-validation
    outer_cv = KFold(n_splits=5, shuffle=True)
    inner_cv = KFold(n_splits=5, shuffle=True)
    linear_search = GridSearchCV(pipe, param_grid=linear_param_grid, cv=inner_cv, n_jobs=-1)
    forest_search = GridSearchCV(pipe, param_grid=forest_param_grid, cv=inner_cv, n_jobs=-1)
    best_model = None
    best_score = float('-inf')
    
    # Run nested cross-validation
    for train_index, test_index in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Use LinearRegression model
        linear_search.fit(X_train, y_train)
        linear_score = linear_search.score(X_test, y_test)
        if linear_score > best_score:
            best_model = linear_search.best_estimator_
            best_score = linear_score
        
        # Use RandomForestRegressor model
        forest_search.fit(X_train, y_train)
        forest_score = forest_search.score(X_test, y_test)
        if forest_score > best_score:
            best_model = forest_search.best_estimator_
            best_score = forest_score
    
    # Return the best model
    return best_model

In [ ]:
def find_optimal_classifier(dataframe, dependent_variable_name, also_drop=()):
    """
    Performs nested cross-validation to identify a classifier that provides the best prediction of the dependent variable.
    
    Args:
    - dataframe: Pandas dataframe with independent and dependent variables as columns and observations as rows.
    - dependent_variable_name: String indicating the name of the column in dataframe that contains the dependent variable.
    
    Returns:
    - clf: Trained scikit-learn classifier that provides the best prediction of the dependent variable.
    """
    
    # Separate independent and dependent variables
    X = dataframe.drop([dependent_variable_name] + list(also_drop), axis=1)
    y = dataframe[dependent_variable_name]
    
    # Define hyperparameters for grid search
    clf1 = RandomForestClassifier()
    params1 = {'n_estimators': [100, 500], 'max_depth': [5, 10]}
    
    clf2 = LogisticRegression()
    params2 = {'penalty': ['l1', 'l2'], 'C': [0.1, 1.0, 10.0]}
    
    # Initialize model list
    models = []
    
    # Create KFold objects for outer and inner loops of cross-validation
    outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    # Perform nested cross-validation
    for i, (train_idx, test_idx) in enumerate(outer_cv.split(X)):
        
        print(f"Fold {i+1}")
            
        X_train_outer, X_test_outer = X.iloc[train_idx], X.iloc[test_idx]
        y_train_outer, y_test_outer = y.iloc[train_idx], y.iloc[test_idx]
        
        best_params = None
        best_score = None
        
        # Perform grid search with the first classifier
        gs1 = GridSearchCV(clf1, params1, cv=inner_cv)
        gs1.fit(X_train_outer, y_train_outer)
        
        # Perform grid search with the second classifier
        gs2 = GridSearchCV(clf2, params2, cv=inner_cv)
        gs2.fit(X_train_outer, y_train_outer)
        
        # Choose the best classifier and its hyperparameters
        if gs1.best_score_ > gs2.best_score_:
            best_params = gs1.best_params_
            best_score = gs1.best_score_
            clf = clf1.set_params(**best_params)
            models.append((clf, best_score))
        else:
            best_params = gs2.best_params_
            best_score = gs2.best_score_
            clf = clf2.set_params(**best_params)
            models.append((clf, best_score))
        
        print(f"Best score: {best_score}")
        print(f"Best params: {best_params}")
        
    # Choose the best classifier across all folds
    best_model = max(models, key=lambda x: x[1])[0]
    
    # Train the chosen classifier on the entire dataset
    best_model.fit(X, y)
    
    return best_model

In [ ]:
pass_model = find_optimal_regressor(T1w_train, "pass", ["rating"])

In [ ]:
predictions = pass_model.predict(T1w_train.drop(["pass", "rating"], axis=1))

In [ ]:
np.sum((predictions > 0.5) != T1w_train["pass"])

In [ ]:
T1w_train["pass"].sum()

In [ ]:
rating_model = find_optimal_regressor(T1w_train, "rating", ["pass"])
rating_predictions = rating_model.predict(T1w_train.drop(["pass", "rating"], axis=1))

In [ ]:
np.min(rating_predictions - T1w_train.rating)

In [ ]:
pyplot.scatter(T1w_train.rating, rating_predictions)

In [ ]:
rating_model

In [ ]:
pass_model

In [ ]:
pass_clf = find_optimal_classifier(T1w_train, "pass", ["rating"])

In [ ]:
pass_clf

In [ ]:
pass_predictions = pass_clf.predict(T1w_train.drop(["pass", "rating"], axis=1))
pyplot.scatter(T1w_train['pass'], pass_predictions)

In [ ]:
T1w_test = pd.read_csv('T1w_test.csv', index_col=0)

In [ ]:
T1w_pred = pass_clf.predict(T1w_test.drop(["pass", "rating"], axis=1))
pyplot.scatter(T1w_test['pass'], T1w_pred)